# Clean polling place data for analysis

In [1]:
%load_ext lab_black

In [125]:
import pandas as pd
import numpy as np

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

In [157]:
src = pd.read_csv(
    "output/polling_places_analysis.csv", dtype={"year": str, "fips": str}
)

In [158]:
src.head()

,fips,place,state,total_reg_voters,mail_ballots_sent,total_votes_cast,poll_place_elect_day,poll_place_early,year
0,0100100000,AUTAUGA COUNTY,AL,43695.0,1329.0,24217.0,18.0,1.0,2020
1,0100300000,BALDWIN COUNTY,AL,176668.0,11147.0,96609.0,50.0,1.0,2020
2,0100500000,BARBOUR COUNTY,AL,17850.0,726.0,9234.0,16.0,1.0,2020
3,0100700000,BIBB COUNTY,AL,15014.0,332.0,9031.0,8.0,1.0,2020
4,0100900000,BLOUNT COUNTY,AL,41927.0,1032.0,25823.0,24.0,1.0,2020


---

### Clean up and categorize place types

In [159]:
src.loc[src["place"].str.contains("COUNTY", case=False), "place_type"] = "County"
src.loc[src["place"].str.contains("PARISH", case=False), "place_type"] = "County"
src.loc[src["place"].str.contains("CITY", case=False), "place_type"] = "City"
src.loc[src["place"].str.contains("TOWN", case=False), "place_type"] = "Town"
src.loc[src["place"].str.contains("VILLAGE", case=False), "place_type"] = "Village"

In [160]:
src.place_type.value_counts()

County     8536
Town       5470
City       2284
Village    1393
Name: place_type, dtype: int64

In [161]:
src["place_clean"] = src["place"].str.title().str.replace(" County", "", regex=False)

In [162]:
src.total_reg_voters.describe()

count     21063
unique    13917
top         0.0
freq        103
Name: total_reg_voters, dtype: object

---

In [163]:
missing_values = [
    "-999999.0",
    "-99.0",
    "-888888.0",
    "-88.0",
    "-999999: Data Not Available",
    "-888888: Not Applicable",
    "",
    "",
    #Minnesota does not have provisional ballots.
]
src = src.replace(missing_values, np.NaN)

In [165]:
src.sort_values("total_votes_cast", ascending=False)

,fips,place,state,total_reg_voters,mail_ballots_sent,total_votes_cast,poll_place_elect_day,poll_place_early,year,place_type,place_clean
8773,2707100000,KOOCHICHING COUNTY,MN,7570,,Minnesota does not have provisional ballots.,,NaN,2016,County,Koochiching
8789,2710300000,NICOLLET COUNTY,MN,21078,,Minnesota does not have provisional ballots.,,NaN,2016,County,Nicollet
8815,2715500000,TRAVERSE COUNTY,MN,2116,,Minnesota does not have provisional ballots.,,NaN,2016,County,Traverse
8816,2715700000,WABASHA COUNTY,MN,13902,,Minnesota does not have provisional ballots.,,NaN,2016,County,Wabasha
8817,2715900000,WADENA COUNTY,MN,8086,,Minnesota does not have provisional ballots.,,NaN,2016,County,Wadena
...,...,...,...,...,...,...,...,...,...,...,...
16153,3503700000,QUAY COUNTY,NM,5979.0,NaN,NaN,NaN,NaN,2012,County,Quay
16157,3504700000,SAN MIGUEL COUNTY,NM,22838.0,NaN,NaN,NaN,NaN,2012,County,San Miguel
16158,3504300000,SANDOVAL COUNTY,NM,86217.0,NaN,NaN,NaN,NaN,2012,County,Sandoval
16164,3505900000,UNION COUNTY,NM,2636.0,0.0,NaN,NaN,NaN,2012,County,Union


In [152]:
# src = src.fillna("").replace("-999999.0", "")
# src = src.fillna("").replace("-99.0", "")
# src = src.fillna("").replace("-88.0", "")
# src = src.fillna("").replace("-888888.0", "")
# src = src.fillna("").replace("-999999: Data Not Available", "")
# src = src.fillna("").replace("-888888: Not Applicable", "")

In [153]:
src[['total_reg_voters', 'mail_ballots_sent',
       'total_votes_cast', 'poll_place_elect_day', 'poll_place_early']] = src[['total_reg_voters', 'mail_ballots_sent',
       'total_votes_cast', 'poll_place_elect_day', 'poll_place_early']].astype(float)

ValueError: could not convert string to float: ''

In [134]:
src[src['total_reg_voters'] > 100].isnull().sum()

TypeError: '>' not supported between instances of 'str' and 'int'